# 실시간 위치 좌표 얻기
---
- 출처: https://parkgihyeon.github.io/project/geocoding-api/

In [137]:
# 현재위치 좌표 얻기
import requests, json

def current_location():
    here_req = requests.get("http://www.geoplugin.net/json.gp")

    if (here_req.status_code != 200):
        print("현재좌표를 불러올 수 없음")
    else:
        location = json.loads(here_req.text)
        crd = {"lat": str(location["geoplugin_latitude"]), "lng": str(location["geoplugin_longitude"])}

    return crd

crd = current_location()
print(crd)

{'lat': '35.8723', 'lng': '128.5924'}


In [123]:
# 가입 없이 좌표->주소 변환
from geopy.geocoders import Nominatim

def geocoding_reverse(lat_lng_str): 
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    address = geolocoder.reverse(lat_lng_str)

    return address

address = geocoding_reverse('35.8723, 128.5924')
print(address)

경상감영1길, 중구, 대구, 41934, 대한민국


In [2]:
# 가입 없이 주소->좌표 변환
from geopy.geocoders import Nominatim

def geocoding(address):
    geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
    geo = geolocoder.geocode(address)
    crd = {"lat": str(geo.latitude), "lng": str(geo.longitude)}

    return crd

crd = geocoding("대구 복현동")
print(crd['lat'])
print(crd['lng'])

35.8968175
128.6194541


# 장소 마크 표시 및 개수 세기
---- 
참고:https://haries.tistory.com/6

# streamlit yolov5 
---
참고 : https://github.com/xugaoxiang/yolov5-streamlit

# streamlit 드론 gps
---
참고: https://github.com/greattoe/ros_tutorial_kr/blob/master/ros1_tutorial/rospy/bebop2/bb2_5_mark_bebop2_on_web.md

# 도로 CCTV 송출
---
참고: https://parkgihyeon.github.io/project/cctv-api/#2%EF%B8%8F%E2%83%A3-cctv-api%EB%A5%BC-%ED%98%B8%EC%B6%9C%ED%95%98%EB%8A%94-%EB%B0%A9%EB%B2%95

# 수정코드
---

In [125]:
# 현재위치 좌표 얻기
import requests, json
import pandas as pd

def current_location():
    here_req = requests.get("http://www.geoplugin.net/json.gp")

    if (here_req.status_code != 200):
        print("현재좌표를 불러올 수 없음")
    else:
        location = json.loads(here_req.text)
        crd = {float(location["geoplugin_latitude"]), float(location["geoplugin_longitude"])}
        crd = list(crd)
        gps = pd.DataFrame( [[crd[1],crd[0]]], columns=['위도','경도'])
    
    return gps

In [128]:
gps = current_location()

In [130]:
gps['위도']

0    35.8723
Name: 위도, dtype: float64

In [132]:
# cctv -----------------------------------------------------------------------------------------------------
import requests
import numpy as np

def get_cctv_url(lat, lng):
    # CCTV 탐색 범위 지정을 위해 임의로 ±1 만큼 가감
    minX = str(lng-1)
    maxX = str(lng+1)
    minY = str(lat-1)
    maxY = str(lat+1)

    # 개인key 입력
    api_call = 'https://openapi.its.go.kr:9443/cctvInfo?' \
               'apiKey=개인key' \
               '&type=ex&cctvType=2' \
               '&minX=' + minX + \
               '&maxX=' + maxX + \
               '&minY=' + minY + \
               '&maxY=' + maxY + \
               '&getType=json'

    w_dataset = requests.get(api_call).json()
    cctv_data = w_dataset['response']['data']

    coordx_list = []
    for index, data in enumerate(cctv_data):
        xy_couple = (float(cctv_data[index]['coordy']),float(cctv_data[index]['coordx']))
        coordx_list.append(xy_couple)

    # 입력한 위경도 좌표에서 가장 가까운 위치에 있는 CCTV를 찾는 과정
    coordx_list = np.array(coordx_list)
    leftbottom = np.array((lat, lng))
    distances = np.linalg.norm(coordx_list - leftbottom, axis=1)
    min_index = np.argmin(distances)

    return cctv_data[min_index]


cctv_data = get_cctv_url(36.58629, 128.186793)
print('CCTV명:', cctv_data['cctvname']) # 가장 가까운 CCTV명
print('CCTV 영상 URL:', cctv_data['cctvurl']) # 가장 가까운 CCTV 영상 URL

KeyError: 'response'

In [134]:
!pip install pygeoip

Defaulting to user installation because normal site-packages is not writeable
